## Basic SetUP

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk

In [ ]:
df = pd.read_csv("MKB_NIOS_data.csv")

In [ ]:
dataset = load_dataset('csv', data_files="MKB_NIOS_data.csv", header=0, split='train')
# dataset = load_dataset('csv',data_files='sample.csv',header = 0, split='train')


In [ ]:
if 'Unnamed: 0' in dataset.column_names:
        dataset = dataset.remove_columns(['Unnamed: 0'])

In [ ]:
dataset = dataset.train_test_split(test_size=0.1, shuffle=False)

In [ ]:
dataset

In [ ]:
model_name = "ai4bharat/IndicBART"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
#

In [ ]:
#source_lang = 'devanagari_kannada' if FLAGS.source_lang.capitalize() == 'Kannada' else FLAGS.source_lang.capitalize()
#target_lang = 'devanagari_kannada' if FLAGS.target_lang.capitalize() == 'Kannada' else FLAGS.target_lang.capitalize()
source_lang = 'Hindi'
target_lang = 'Sanskrit'
language_code = {'Hindi':'hi', 'Sanskrit':'sa', 'Kannada':'kn'}
#s_lang = language_code[FLAGS.source_lang.capitalize()]
#t_lang = language_code[FLAGS.target_lang.capitalize()]
s_lang = 'hi'
t_lang = 'sa'

In [ ]:
max_input_length = 128
max_target_length = 128

## POS SetUP

In [ ]:
from flair.models import SequenceTagger
from flair.data import Sentence

In [ ]:
# from flair_mod.data_mod import Sentence

In [ ]:
model = SequenceTagger.load('resources/taggers/final-model.pt')

In [ ]:
from sklearn.preprocessing import LabelEncoder
import re

In [ ]:
tags = ['<unk>','PSP','NN', 'VM','NNP','SYM','VAUX','JJ','NNPC',
 'PRP','CC','NNC','QC','NST','DEM','RP','QF','NEG','RB','QCC',
 'QO','INTF','JJC','WQ','RDP','UNK','PRPC','NSTC','RBC','QFC','CCC','INJ']

In [ ]:
le = LabelEncoder()
le = le.fit(tags)

In [ ]:
word_tokenize("आप सब lockdown में इस ‘मन की बात’ को सुन रहे हैं |")

In [ ]:
def pos_tag(sentence,tokenizer):
    # check_sent[0] = sentence
    sent_tok= sentence.split(" ")
    sent = Sentence(sentence)
    model.predict(sent)
    tags = sent.get_labels()
    # print(tags)
    
    pos_tag_val = re.findall("'+([A-Z]{2,})'",str(tags))
    encoded_tags = list(le.transform(pos_tag_val))
    # encoded_tags.extend([4401,4407])
    encoded_tags = [i*3 for i in encoded_tags]
    # encoded_tags.insert(0,2)
    # encoded_tags.insert(len(encoded_tags),3)
    #Test -- Tokinzer
    final_encoded_pos = []
    token_nlp = tokenizer(sent_tok)
    for i in range(len(sent_tok)):
        if len(token_nlp.input_ids[i]) == 3:
            final_encoded_pos.append(encoded_tags[i])
            # try:
            #     final_encoded_pos.append(encoded_tags[i])
            # except:
            #     print("Encoded_Tags",len(encoded_tags),encoded_tags)
            #     print("Token NLP", len(token_nlp.input_ids), token_nlp.input_ids)
            #     print("Sent TOkens",len(sent_tok),sent_tok)
            #     print(sentence)
        else:
            num_divided_words = len(token_nlp.input_ids[i]) - 2
            final_encoded_pos += num_divided_words * [encoded_tags[i]]
            # try:
            #     final_encoded_pos += num_divided_words * [encoded_tags[i]]
            # except:
            #     print("Encoded_Tags",len(encoded_tags),encoded_tags)
            #     print("Token NLP", len(token_nlp.input_ids), token_nlp.input_ids)
            #     print("Sent TOkens",len(sent_tok),sent_tok)
            #     print(sentence)

    final_encoded_pos.insert(0,2)
    final_encoded_pos.extend([4401,4407,3])
            
            

    return final_encoded_pos

## Tokenization

In [ ]:

def preprocess_function(examples):

        sentences= examples[source_lang]
        inputs = [example + ' </s>' + f' <2{s_lang}>' for example in examples[source_lang]]
        targets = [f'<2{t_lang}> ' + example + ' </s>' for example in examples[target_lang]]
        # print(inputs)
        # print(targets)
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

        with tokenizer.as_target_tokenizer():
            labels = tokenizer(targets, max_length=max_target_length, truncation=True)
        


        model_inputs['labels'] = labels['input_ids']
        print(type(labels))
        print(type(model_inputs))
        print(model_inputs.keys())
        print(len(inputs))
        lis_pos = []
        # pos_tagger = train_pos()

        for i in tqdm(range(len(inputs))):
            # print(len(model_inputs['input_ids'][i]),model_inputs['input_ids'][i])
            # print
            tag_val = pos_tag(sentences[i],tokenizer)
            # print("len",len(tag_val),len(model_inputs['input_ids'][i]))
            # print("--_--"*25)
            # print('val',sentences[i], tag_val,model_inputs['input_ids'][i])
            lis_pos.append(tag_val)
            print("Checking",len(tag_val),len(model_inputs['input_ids'][i]))
            # if len(model_inputs['input_ids'][i]) != len(pos_tag):
            #     # c+=1
            #     pos_update_val = []
            #     # update_pos = {}
            #     num = len(model_inputs['input_ids'][i]) - 2
            #     # print(i)
            #     pos_update_val += num * [6]
            #     pos_update_val.insert(0,2)
            #     pos_update_val.insert(len(pos_update_val),1)
            #     # print("Modified",len(pos_update_val),pos_update_val)
            #     # update_pos["pos"] = pos_update
            #     lis_pos.append(pos_update_val)
            # else:
            #     # print('Original',len(pos_tag),pos_tag)
            #     lis_pos.append(pos_tag) 
        # for i in tqdm(range(len(inputs))):
        #     print(i)
        #     lis_pos.append(pos_tag_fn(inputs[i],pos_tagger,tokenizer))
            #print(i,val)

        model_inputs.data['pos'] = lis_pos
        # print(model_inputs)
        # model_inputs.concat(pos_dict)
        
        #a.append(model_inputs)
        print("----Complete----")
        return model_inputs

In [ ]:
batch_size = 16

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
#NIos + MKB--12K approx sentence-- 56mins


## HuggingFace Dataset Save

In [ ]:
tokenized_datasets.save_to_disk("/nlsasfs/home/ttbhashini/prathosh/divyanshu/pos_hindi")

In [ ]:
reloaded_dataset = load_from_disk("/nlsasfs/home/ttbhashini/prathosh/divyanshu/pos_hindi")

In [ ]:
reloaded_dataset